In [1]:
import cv2                 
import numpy as np         
import os                  
from random import shuffle 
from tqdm import tqdm  
import pandas as pd

TRAIN_DIR = './lungs_images'
TEST_DIR = './test1'
IMG_SIZE = 50
LR = 1e-2

MODEL_NAME = 'lung_cancer-{}-{}.model'.format(LR, 'lungs-basic-test') 
labels = {1:'Atelectasis', 2: 'Cardiomegaly', 3: 'Effusion', 4: 'Infiltration', 5: 'Mass', 6: 'Nodule', 7: 'Pneumonia', 8:
'Pneumothorax', 9: 'Consolidation', 10: 'Edema', 11: 'Emphysema', 12: 'Fibrosis', 13:
'Pleural_Thickening', 14: 'Hernia',15:'No Finding'}
labels = {v: k for k, v in labels.iteritems()}


In [2]:
ds = pd.read_csv('Data_Entry_2017.csv')
ds.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [3]:
def split_map(val):
    new_val = []
    val = val.split('|')
    for i in val:
        new_val.append(labels[i])
    new_val = '|'.join(str(e) for e in new_val)
    return new_val
def split_map_arr(val):
    new_val = []
    arr = np.zeros((15,), dtype=int)
    val = val.split('|')
    for i in val:
        ind = labels[i]-1
        arr[ind] = 1
        
    return arr



In [4]:
value_r={}
for (i,v) in ds['Finding Labels'].iteritems():
    str_val = str(v)
    val = split_map(str_val)
    value_r[str_val]  = val
    
# ds['Finding Labels'] = ds['Finding Labels'].map(value_r)

In [5]:
ds1 = ds[['Image Index', 'Finding Labels']]

In [6]:
value_r1={}
for (i,v) in ds1['Finding Labels'].iteritems():
    str_val = str(v)
    val = split_map_arr(str_val)
    value_r1[str_val]  = val

ds1['Finding Labels'] = ds1['Finding Labels'].map(value_r1)


/home/mg/anaconda3/envs/sarcasm/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [7]:
ds1.head()

,Image Index,Finding Labels
0,00000001_000.png,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,00000001_001.png,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
2,00000001_002.png,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,00000002_000.png,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
4,00000003_000.png,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]"


In [8]:
def create_label(img):
    vs = ds1.loc[ds1['Image Index'] == img]
    return vs.iloc[0]['Finding Labels']

def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = create_label(img)
        
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [9]:
# train_data = create_train_data()
train_data = np.load('train_data.npy')

In [10]:
train_data

array([[array([[48, 20, 19, ..., 29, 29, 33],
       [46, 19, 18, ..., 20, 21, 23],
       [44, 18, 16, ..., 18, 28, 23],
       ...,
       [20, 16, 14, ..., 17, 20, 63],
       [19, 16, 14, ..., 19, 17, 60],
       [23, 20, 16, ..., 19, 17, 55]], dtype=uint8),
        array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])],
       [array([[0, 0, 0, ..., 0, 0, 2],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
        array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
       [array([[  2,   2,   2, ...,   9,  12,  19],
       [  2,   2,   1, ...,   7,   8,  11],
       [  1,   2,   2, ...,   5,   6,   8],
       ...,
       [106, 115, 121, ...,  91, 105, 112],
       [105, 116, 122, ...,  88, 104, 131],
       [107, 114, 127, ...,  75, 132, 109]], dtype=uint8),
        array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
       ...,
       [ar

In [11]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 15, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

/home/mg/anaconda3/envs/sarcasm/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [12]:

model.load(MODEL_NAME)
print('model loaded!')


INFO:tensorflow:Restoring parameters from /home/mg/Machine_learning/lung_cancer-0.01-lungs-basic-test.model
model loaded!


In [13]:
train = train_data[:-10000]
test = train_data[-10000:]

In [14]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]


In [15]:
# model.fit({'input': X}, {'targets': Y}, n_epoch=3, validation_set=({'input': test_x}, {'targets': test_y}), 
#     snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

In [16]:
# model.save(MODEL_NAME)

In [17]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data1.npy', testing_data)
    return testing_data

In [18]:
import matplotlib.pyplot as plt

# if you need to create the data:
test_data = process_test_data()
# if you already have some saved:
print test_data
# test_data = np.load('test_data.npy')



for data in tqdm(test_data):
        img_num = data[1]
        img_data = data[0]
        orig = img_data
        data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
        model_out = model.predict([data])[0]

  0%|          | 0/1 [00:00<?, ?it/s]

[[array([[  4,   4,   4, ...,   3,   3,   3],
       [  4,   4,   4, ...,  30,  14,   3],
       [  4,   4,   4, ...,  52,  41,  22],
       ...,
       [ 26, 139, 168, ...,  25,   4,   7],
       [ 39, 160, 193, ...,  29,  10,   6],
       [184, 216, 226, ...,  63,  26,   9]], dtype=uint8), '00000324_001']]


100%|██████████| 1/1 [00:00<00:00,  1.44it/s]


In [19]:
model_out

array([0.08071883, 0.01746977, 0.09639819, 0.13747564, 0.0418332 ,
       0.04875651, 0.01125745, 0.0396365 , 0.03097828, 0.01728884,
       0.01541501, 0.01433229, 0.02423817, 0.00164826, 0.42255297],
      dtype=float32)